# Конспект по теме: "Временные ряды для аналитика"

Этот конспект предназначен для аналитиков (новичков и среднего уровня), которые хотят освоить анализ временных рядов с помощью Python и библиотеки Pandas. Включены основные понятия, методы, примеры на реальных данных и практические задания.

## 1. Что такое временной ряд и где применяется

**Временной ряд** — это последовательность данных, упорядоченных по времени. Каждое наблюдение имеет временную метку (дату или временной интервал).

### Примеры временных рядов:
- Ежедневные продажи товара
- Часовой трафик на сайте
- Ежемесячная инфляция
- Курсы валют
- Показания датчиков (температура, давление)

### Основные компоненты временного ряда:
1. **Тренд (Trend)** — долгосрочное систематическое изменение (рост или падение).
2. **Сезонность (Seasonality)** — регулярные колебания, повторяющиеся с фиксированной периодичностью (день, неделя, месяц, год).
3. **Цикличность (Cycles)** — длительные колебания, не имеющие фиксированного периода (например, экономические циклы).
4. **Шум (Noise)** — случайные, непредсказуемые флуктуации.

### Модели декомпозиции:
- **Аддитивная модель**: `Y(t) = T(t) + S(t) + C(t) + E(t)`
- **Мультипликативная модель**: `Y(t) = T(t) × S(t) × C(t) × E(t)`

Выбор модели зависит от характера данных. Если амплитуда сезонных колебаний постоянна — аддитивная. Если амплитуда растёт с уровнем ряда — мультипликативная.

### Где применяется:
- Прогнозирование продаж и спроса
- Финансовый анализ (курсы акций, криптовалют)
- Мониторинг бизнес-метрик (DAU, MAU, CTR)
- Прогноз погоды и климата
- Аналитика IoT-устройств

## 2. Дата и время в Pandas

Pandas предоставляет мощные инструменты для работы с временными данными.

### Основные типы:
- `Timestamp` — момент времени (аналог `datetime.datetime`)
- `Timedelta` — разница между двумя временными метками
- `Period` — временной интервал (например, месяц, квартал)
- `DatetimeIndex` — индекс на основе временных меток

### Основные функции:

```python
import pandas as pd

# Парсинг даты при чтении
df = pd.read_csv('data.csv', parse_dates=['date'], index_col='date')

# Преобразование в datetime
df['date'] = pd.to_datetime(df['date'])

# Установка индекса
df.set_index('date', inplace=True)

# Генерация диапазона дат
dates = pd.date_range('2023-01-01', periods=100, freq='D')

# Фильтрация по дате
df['2023-01-01':'2023-01-31']
df[df.index.year == 2023]

# Извлечение компонентов даты
df.index.year
df.index.month
df.index.dayofweek  # 0 = понедельник
df.index.quarter
```

## 3. Ресемплинг и скользящее среднее

### Ресемплинг (Resampling)

Изменение частоты временного ряда: агрегация (downsampling) или интерполяция (upsampling).

```python
# Пример: ежедневные данные → ежемесячные
monthly_sales = daily_sales.resample('M').sum()  # сумма за месяц
monthly_avg = daily_sales.resample('M').mean()   # среднее за месяц

# Частоты: 'D' (день), 'W' (неделя), 'M' (месяц), 'Q' (квартал), 'Y' (год)
```

### Скользящее среднее (Moving Average)

Метод сглаживания для выявления тренда.

Формула:

$$ \text{MA}_t = \frac{1}{n} \sum_{i=0}^{n-1} x_{t-i} $$

```python
# Скользящее среднее за 7 дней
df['MA_7'] = df['value'].rolling(window=7).mean()

# Скользящее среднее за 30 дней
df['MA_30'] = df['value'].rolling(window=30).mean()
```

### Экспоненциальное сглаживание

Более весомые значения даются последним наблюдениям.

Формула:

$$ \hat{y}_t = \alpha \cdot x_t + (1 - \alpha) \cdot \hat{y}_{t-1} $$

```python
df['EMA'] = df['value'].ewm(alpha=0.1).mean()
```

## 4. Выявление тренда и сезонности

### Визуальный анализ
- График временного ряда
- Диаграммы размаха (boxplot) по месяцам/дням недели
- Скользящие средние

### Статистические методы

#### Декомпозиция

Разделение ряда на компоненты: тренд, сезонность, остатки.

```python
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df['value'], model='additive', period=12)  # для месячных данных
result.plot()
plt.show()
```

#### Тест на стационарность

**Тест Дики-Фуллера (ADF)** — проверка на стационарность.

```python
from statsmodels.tsa.stattools import adfuller

result = adfuller(df['value'])
print('p-value:', result[1])
# p-value < 0.05 → ряд стационарен
```

Если ряд нестационарен — применяют дифференцирование:

```python
df['diff'] = df['value'].diff().dropna()
```

## 5. Простое прогнозирование

### Наивные модели
- **Наивное предсказание**: завтра = сегодня → `y_hat = y_t`
- **Сезонно-наивное предсказание**: завтра = вчера в том же сезоне → `y_hat = y_{t-s}`

### Модель ARIMA

ARIMA(p, d, q) — авторегрессионная интегрированная модель скользящего среднего.
- `p` — порядок авторегрессии (AR)
- `d` — порядок дифференцирования (I)
- `q` — порядок скользящего среднего (MA)

```python
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(train_data, order=(1, 1, 1))
fitted_model = model.fit()
forecast = fitted_model.forecast(steps=30)
```

### Модель Prophet (от Facebook)

Подходит для рядов с трендом, сезонностью и праздниками.

```python
from prophet import Prophet

df_prophet = df.reset_index().rename(columns={'date': 'ds', 'value': 'y'})
model = Prophet()
model.fit(df_prophet)
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)
```

## 6. Примеры на реальных данных

### Пример 1: Продажи товара (ежедневные данные)

```python
# Загрузка данных (пример)
sales = pd.read_csv('sales_data.csv', parse_dates=['date'], index_col='date')

# Ресемплинг в ежемесячные продажи
monthly_sales = sales.resample('M').sum()

# Скользящее среднее
monthly_sales['MA_3'] = monthly_sales['sales'].rolling(3).mean()

# Декомпозиция
decomp = seasonal_decompose(monthly_sales['sales'], model='additive', period=12)
decomp.plot()
plt.show()
```

### Пример 2: Трафик на сайте (ежечасные данные)

```python
traffic = pd.read_csv('traffic.csv', parse_dates=['timestamp'], index_col='timestamp')

# Выделение часа и дня недели
traffic['hour'] = traffic.index.hour
traffic['dayofweek'] = traffic.index.dayofweek

# Boxplot по часам
traffic.boxplot(column='visits', by='hour', figsize=(10, 6))
plt.show()
```

### Источники данных:
- [Kaggle: Daily Retail Sales](https://www.kaggle.com/datasets)
- [UCI: Air Quality](https://archive.ics.uci.edu/ml/datasets)
- [FRED: Макроэкономические данные](https://fred.stlouisfed.org/)

## 7. Практические задания

Выполните следующие задания в Jupyter Notebook. Используйте открытые датасеты (например, с Kaggle или UCI).

### Задание 1: Анализ продаж
1. Загрузите датасет с ежедневными продажами товара.
2. Преобразуйте столбец с датой в `datetime` и установите его в качестве индекса.
3. Постройте график продаж.
4. Ресемплируйте данные в ежемесячные суммы.
5. Постройте 12-месячное скользящее среднее.
6. Сделайте вывод о наличии тренда.

### Задание 2: Выявление сезонности
1. Загрузите данные с ежечасным трафиком на сайт.
2. Постройте boxplot по часам суток.
3. Постройте boxplot по дням недели.
4. Сделайте вывод о пиках трафика.
5. Примените декомпозицию (период = 24).

### Задание 3: Прогнозирование с помощью ARIMA
1. Возьмите временной ряд из задания 1.
2. Проверьте стационарность с помощью теста ADF.
3. Если нужно — примените дифференцирование.
4. Постройте модель ARIMA(1,1,1) и сделайте прогноз на 6 месяцев вперёд.
5. Визуализируйте прогноз.

### Задание 4: Прогнозирование с помощью Prophet
1. Установите библиотеку `prophet`.
2. Подготовьте данные в формате `ds` (дата) и `y` (значение).
3. Обучите модель Prophet.
4. Сделайте прогноз на 90 дней.
5. Постройте график прогноза с доверительным интервалом.

### Задание 5: Сравнение моделей
1. Для одного и того же ряда постройте прогнозы: наивный, ARIMA, Prophet.
2. Разделите данные: 80% — обучение, 20% — тест.
3. Оцените модели с помощью MAE и RMSE.
4. Составьте таблицу сравнения.
5. Напишите вывод: какая модель лучше и почему.

In [ ]:
# Пример кода для начала работы
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-v0_8')